In [8]:
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.firefox.options import Options
import warnings
warnings.filterwarnings('ignore');

In [2]:
def create_firefox_webdriver(headless=False):
    # Set options for WebDriver
    options = Options()
    options.headless = headless

    # Create a new instance of the Firefox driver
    driver = webdriver.Chrome(options=options)

    return driver

driver = create_firefox_webdriver(headless=False)

In [3]:
##Load hotel list
hotels = pd.read_excel('Hotels in Ayodhya.xlsx')
df1=hotels.copy()

##### Extract Data

In [53]:
hotel_data=[]
meta_data={}
for idx,value in tqdm(df1.iterrows(),desc='Extracting.............'):
        link='https:'+value['link']
        driver.refresh()
        driver.get(link)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        fclt_div = soup.find('div', class_='appendTop52')
        meta_data['name']=value['Hotel_Name']

    
        meta_data['aminity']=[]
        try:
            time.sleep(1)
            _divs = fclt_div.find_all('div', class_='fcltyLst_item')
            for _div in _divs:
                meta_data['aminity'].append(_div.text+' *')
        except Exception as e:
            print(idx," ",e)
            meta_data['aminity']=None


        meta_data['rules']=[]
        try:
            rule_div = soup.find_all('div', class_='htlRules__list--item')[1:]
            for element in rule_div:
                meta_data['rules'].append(element.text.strip()+" *")
        except Exception as e:
            print(idx," ",e)
            meta_data['rules']=None


        meta_data['reviews']=[]
        try:
            review_div = soup.find('div', class_='_RatingReview')
            if review_div:
                divs = review_div.find_all('div', class_='reviewBox')
                if len(divs)>1:
                    divs=divs[1:]
                for div in (divs):
                    meta_data['reviews'].append(div.text+" *")
        except Exception as e:
            print(idx,"  ",e)
            meta_data['reviews']=None
            meta_data={}


        hotel_data.append(meta_data)
        meta_data={}

Extracting.............: 103it [15:41,  9.14s/it]


In [35]:
temp=pd.DataFrame.from_records(hotel_data)

In [54]:
ht=hotel_data

In [56]:
for hotel in ht:
    hotel['aminity']=" ".join(hotel['aminity'])
    hotel['rules']=" ".join(hotel['rules'])
    hotel['reviews']=" ".join(hotel['reviews'])

In [59]:
temp=pd.DataFrame.from_records(ht)

##### Review sentiment analysis

In [77]:
from transformers import pipeline
pipe= pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    return_all_scores=True
)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [78]:
rslt=pipe('im felling very dull today')

In [87]:
rslt[0][2]

{'label': 'negative', 'score': 0.733554482460022}

In [98]:
temp['positive']=None
temp['negative']=None
temp['neutral']=None

null_reviews_idx=temp.loc[temp['reviews']==""].index

def calculate_mean(value):
    if not value:
        return None
    else:
        total=sum(value)
        mean=total/len(value)
        return mean


for i,review in tqdm(enumerate(temp['reviews']),desc='processing'):
    pos=[]
    neu=[]
    neg=[]
    if i not in null_reviews_idx:
        splitted_review=review.split("*")
        total_reviews=len(splitted_review)
        temp.loc[i,'total_reviews']=total_reviews
        for each in splitted_review:
            result=pipe(each)
            pos.append(result[0][0]['score'])
            neu.append(result[0][1]['score'])
            neg.append(result[0][1]['score'])
        temp.loc[i,'positive']=calculate_mean(pos)*100
        temp.loc[i,'neutral']=calculate_mean(neu)*100
        temp.loc[i,'negative']=calculate_mean(neg)*100
    
        
    

processing: 103it [01:31,  1.13it/s]


In [123]:
temp.head()

,name,aminity,rules,reviews,positive,negative,neutral,total_reviews
0,Apoorva Guest House,Highlighted Amenities- Free Parking- Power Bac...,"Aadhar, Driving License and Govt. ID are accep...",,0.000000,0.000000,0.000000,NaN
1,Hotel Hanuman Ji,Highlighted Amenities- Fireplace- Smoking Room...,Outside food is not allowed * Unmarried couple...,Excellent Stay Rated 5.0 by PRADEEP MANOHAR K...,65.303374,13.779504,13.779504,7.0
2,OYO Home Siya Homestay,Popular Amenities- Caretaker- Luggage Storage-...,"Passport, Aadhar, Driving License and Govt. ID...",,0.000000,0.000000,0.000000,NaN
3,Hotel Shri Ram Bhavan Seeta Swayamvar Marriage...,Highlighted Amenities- Water Sports- Spa- Rest...,Guests below 18 years of age are not allowed a...,Room Rated 4.0 by Sanjeev Kumar Saxena . Oct...,48.411285,26.455422,26.455422,7.0
4,Life Point grand,Highlighted Amenities- Restaurant- Fireplace- ...,Guests below 18 years of age are not allowed a...,Excellent Stay Rated 5.0 by Manish Kumar . N...,69.529149,15.900148,15.900148,2.0


In [121]:
temp['positive'].fillna(0,inplace=True)
temp['negative'].fillna(0,inplace=True)
temp['neutral'].fillna(0,inplace=True)

In [122]:
temp.to_excel('Hotel-meta-data_colllected.xlsx',index=False)